<a href="https://colab.research.google.com/github/g-arjuna/raft/blob/main/revised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    # This library is our indicator that the required installs
    # need to be done.
    import pyreft

except ModuleNotFoundError:
    !pip install git+https://github.com/stanfordnlp/pyreft.git
    !pip install nnsight

In [10]:
import torch, transformers, pyreft,csv
device = "cuda"
# Define the prompt template
prompt_no_input_template = """<s>[INST] <<SYS>>
You are a computer networking expert specialized in Cisco ACI ( Application Specific Infrastructure ).
<</SYS>>

%s [/INST]
"""

# Load the model
model_name_or_path = "meta-llama/Llama-3.2-3B"
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name_or_path, torch_dtype=torch.bfloat16, device_map=device)

# Load the tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name_or_path, model_max_length=2048,
    padding_side="right", use_fast=False)

# Set pad_token as eos_token
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [4]:
def load_training_examples(filename='questions_answers.csv'):
    training_examples = []
    try:
        with open(filename, mode='r', newline='', encoding='utf-8') as file:
            reader = csv.reader(file)
            next(reader)  # Skip the header row
            for row in reader:
                # Check if the answer field is not empty
                if len(row) >= 2 and row[1].strip():
                    training_examples.append([row[0], row[1]])  # Assuming row format is correct
                else:
                    print(f"Skipped row with empty answer: {row}")
    except FileNotFoundError:
        print(f"The file {filename} was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
    return training_examples

In [7]:
training_examples = load_training_examples()

print(training_examples[:5])
##### Fine-tune the model
output_csv_file = 'generated_answers.csv'

[['What is the primary purpose of using Cisco ACI with container orchestrator engines like Kubernetes?', 'The primary purpose of using Cisco ACI with container orchestrator engines like Kubernetes is to provide network connectivity, IP address management, and security policies to containerized workloads.'], ['How do Calico, Kube-Router, and Cilium differ in their approach to networking modes?', 'Calico supports two main network modes: direct container routing (no overlay transport protocol) or network overlay using VXLAN or IPinIP (default) encapsulations. Kube-Router also supports two main network modes: direct container routing or network overlay using PinIP (default) encapsulations. Cilium, on the other hand, provides networking, security, and observability for cloud-native environments through its use of eBPF technology.'], ["What is the difference between Calico's direct routing approach and Kube-Router's direct routing approach?", "The direct routing approaches in Calico and Kube

In [15]:
# Set up the ReFT config
reft_config = pyreft.ReftConfig(representations={
    "layer": 8, "component": "block_output",
    "low_rank_dimension": 4,
    "intervention": pyreft.LoreftIntervention(embed_dim=model.config.hidden_size,
    low_rank_dimension=4)})
reft_model = pyreft.get_reft_model(model, reft_config)
reft_model.set_device(device)
reft_model.print_trainable_parameters()

trainable intervention params: 24,580 || trainable model params: 0
model params: 3,212,749,824 || trainable%: 0.000765076689643918


In [16]:
# Create data module
data_module = pyreft.make_last_position_supervised_data_module(
    tokenizer, model, [prompt_no_input_template % e[0] for e in training_examples],
    [e[1] for e in training_examples])
print(data_module)

{'train_dataset': Dataset({
    features: ['input_ids', 'intervention_locations', 'labels'],
    num_rows: 1924
}), 'eval_dataset': None, 'data_collator': ReftDataCollator(data_collator=DataCollatorForSeq2Seq(tokenizer=PreTrainedTokenizerFast(name_or_path='meta-llama/Llama-3.2-3B', vocab_size=128000, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|end_of_text|>', 'pad_token': '<|end_of_text|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, l

In [17]:
# Define training arguments
training_args = transformers.TrainingArguments(
    num_train_epochs=100.0, output_dir="./tmp", per_device_train_batch_size=10,
    learning_rate=4e-3, logging_steps=20)

In [18]:
# Train the model
trainer = pyreft.ReftTrainerForCausalLM(
    model=reft_model, tokenizer=tokenizer, args=training_args, **data_module)
_ = trainer.train()

TypeError: Object of type type is not JSON serializable

In [ ]:
ipython


NameError: name 'ipython' is not defined